In [5]:
from pathlib import Path

import numpy as np
import pandas as pd

from src.paths import *

In [6]:
%load_ext autoreload
%autoreload 2

In [9]:
!ls $raw_data_cache/CMAPSSData

Damage Propagation Modeling.pdf test_FD002.txt
RUL_FD001.txt                   test_FD003.txt
RUL_FD002.txt                   test_FD004.txt
RUL_FD003.txt                   train_FD001.txt
RUL_FD004.txt                   train_FD002.txt
readme.txt                      train_FD003.txt
test_FD001.txt                  train_FD004.txt


In [21]:
!cat $raw_data_cache/CMAPSSData/readme.txt

Data Set: FD001
Train trjectories: 100
Test trajectories: 100
Conditions: ONE (Sea Level)
Fault Modes: ONE (HPC Degradation)

Data Set: FD002
Train trjectories: 260
Test trajectories: 259
Conditions: SIX 
Fault Modes: ONE (HPC Degradation)

Data Set: FD003
Train trjectories: 100
Test trajectories: 100
Conditions: ONE (Sea Level)
Fault Modes: TWO (HPC Degradation, Fan Degradation)

Data Set: FD004
Train trjectories: 248
Test trajectories: 249
Conditions: SIX 
Fault Modes: TWO (HPC Degradation, Fan Degradation)



Experimental Scenario

Data sets consists of multiple multivariate time series. Each data set is further divided into training and test subsets. Each time series is from a different engine � i.e., the data can be considered to be from a fleet of engines of the same type. Each engine starts with different degrees of initial wear and manufacturing variation which is unknown to the user. This wear and variation is considered normal, i.e., it is not considered a fault condition. Th

In [62]:
parent_dir = raw_data_cache / "CMAPSSData"

train_df = []
test_df = []
labels = []

for fn in sorted(list(parent_dir.iterdir())):
    print(fn.name)
    if "train" in fn.name:
        df = pd.read_table(fn, sep=" ", header=None)
        train_df.append(df)

    if "test" in fn.name:
        df = pd.read_table(fn, sep=" ", header=None)
        test_df.append(df)

    if "RUL" in fn.name:
        label = pd.read_table(fn, sep=" ", header=None)
        labels.append(label)

Damage Propagation Modeling.pdf
RUL_FD001.txt
RUL_FD002.txt
RUL_FD003.txt
RUL_FD004.txt
readme.txt
test_FD001.txt
test_FD002.txt
test_FD003.txt
test_FD004.txt
train_FD001.txt
train_FD002.txt
train_FD003.txt
train_FD004.txt


In [63]:
feature_names = ["unit_no", "t_cycles", "setting_1", "setting_2", "setting_3"] + [
    f"sensor_{i}" for i in range(21)
]

shape_offset = 0
for i in range(len(train_df)):
    df = train_df[i].copy(deep=True)
    df.drop([26, 27], axis=1, inplace=True)
    df.columns = feature_names
    
    max_life = df.groupby('unit_no')['t_cycles'].max()
    max_life.rename('max_life', inplace=True)
    df = df.join(max_life, how='left', on='unit_no')
    df = df.assign(rul = df.max_life - df.t_cycles)
    
    # reassign unit numbers
    df = df.assign(unit_no = df.unit_no+shape_offset)
    print(shape_offset)
    shape_offset += df.unit_no.nunique()
    train_df[i] = df
    

0
100
360
460


In [64]:
df = pd.concat(train_df, ignore_index=True, axis=0)


In [77]:
feature_names = [x for x in feature_names if x not in ['unit_no','max_life','rul']] 

In [78]:
feature_names

['t_cycles',
 'setting_1',
 'setting_2',
 'setting_3',
 'sensor_0',
 'sensor_1',
 'sensor_2',
 'sensor_3',
 'sensor_4',
 'sensor_5',
 'sensor_6',
 'sensor_7',
 'sensor_8',
 'sensor_9',
 'sensor_10',
 'sensor_11',
 'sensor_12',
 'sensor_13',
 'sensor_14',
 'sensor_15',
 'sensor_16',
 'sensor_17',
 'sensor_18',
 'sensor_19',
 'sensor_20']

## Modeling

In [86]:
import torch.nn.functional as F
import torch.nn as nn
import torch

In [ ]:
# class Model0(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.i_h = nn.Embedding(nv,nh)  # green arrow
#         self.h_h = nn.Linear(nh,nh)     # brown arrow
#         self.h_o = nn.Linear(nh,nv)     # blue arrow
#         self.bn = nn.BatchNorm1d(nh)
        
#     def forward(self, x):
#         h = self.bn(F.relu(self.h_h(self.i_h(x[:,0]))))
#         if x.shape[1]>1:
#             h = h + self.i_h(x[:,1])
#             h = self.bn(F.relu(self.h_h(h)))
#         if x.shape[1]>2:
#             h = h + self.i_h(x[:,2])
#             h = self.bn(F.relu(self.h_h(h)))
#         return self.h_o(h)

In [85]:
class Model0(nn.Module):
    def __init__():
        super(self, nn.Module).__init__()
        self.linear1 = nn.Linear(25,25)
        self.linear_beta = nn.Linear(25,1)
        self.linear_alpha = nn.Linear(25,1)
        self.bn = nn.BatchNorm1d(25)
        self.softplus = nn.Softplus(1)
    def forward(x):
        h = self.bn(F.relu(self.linear1(x[:,0])))
        if x.shape[1]>1:
            h = h + x[:,1]
            h = self.bn(F.relu(self.linear1(x[:,1])))
        if x.shape[1]>2:
            h = h + x[:,2]
            h = self.bn(F.relu(self.linear1(x[:,2])))
        return self.softplus(self.linear_beta(h)), torch.exp(self.linear_alpha(h))
            
        
        

In [ ]:
def weibull_loglikelihood_continuous(a_, b_, y_, u_,name=None):
    ya = tf.div(y_+1e-35,a_)
    return(
        tf.mul(u_,
               tf.log(b_)+tf.mul(b_,tf.log(ya))
              )- 
        tf.pow(ya,b_)
    )

def weibull_loglikelihood_discrete(a_, b_, y_, u_, name=None):
    with tf.name_scope(name):
        hazard0 = tf.pow(tf.div(y_+1e-35,a_),b_) 
        hazard1 = tf.pow(tf.div(y_+1,a_),b_)
    return(tf.mul(u_,tf.log(tf.exp(hazard1-hazard0)-1.0))-hazard1)

In [ ]:
def weibull_loss()